# Quantitative Value Investing Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
from statistics import mean #The statistics's mean function

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world we would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

We will not be paying for access to the index provider's API rather there is a static version of the S&P 500 constituents available here.
[Click this link to download them now](http://nickmccullum.com/algorithmic-trading-python/sp_500_stocks.csv). 

Move this file to where this Jupyter Notebook file is located.

Now we import these stocks.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now we import our IEX Cloud API token.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to local Git repository. We'll be using a sandbox API token in this project, which means that the data we'll use is randomly-generated.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [ ]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'A(SEOTEA/ QK)NGASGBLLTEN RDLAS MC',
 'calculationPrice': 'tops',
 'open': 0,
 'openTime': None,
 'openSource': 'ioilafcf',
 'close': 0,
 'closeTime': None,
 'closeSource': 'acilioff',
 'high': 0,
 'highTime': None,
 'highSource': None,
 'low': 0,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 141.83,
 'latestSource': 'IEX real time price',
 'latestTime': '12:59:42 PM',
 'latestUpdate': 1679527385408,
 'latestVolume': None,
 'iexRealtimePrice': 138.77,
 'iexRealtimeSize': 83,
 'iexLastUpdated': 1627071743099,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 143.56,
 'previousVolume': 65431353,
 'change': -0.86,
 'changePercent': -0.00618,
 'volume': None,
 'iexMarketPercent': 0.02343613505340034,
 'iexVolume': 694404,
 'avgTo

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio. Here is an example.

In [4]:
pe_ratio = data['peRatio']
pe_ratio

31.58

## Executing A Batch API Call & Building Our DataFrame

Batch API calls are one of the easiest ways to improve the performance of code.

This is because HTTP requests are typically one of the slowest components of a script.

Batch API calls are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [5]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,150.015,50.99,N/A
1,AAL,21.670,-1.46,N/A
2,AAP,209.470,22.29,N/A
3,AAPL,140.850,31.55,N/A
4,ABBV,116.020,40.4,N/A
...,...,...,...,...
500,YUM,116.240,31.28,N/A
501,ZBH,164.110,61.82,N/A
502,ZBRA,547.740,44.99,N/A
503,ZION,54.000,6.65,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,657.850,5.06,N/A
1,CINF,119.060,6.36,N/A
2,ZION,54.000,6.65,N/A
3,AFL,54.200,7.28,N/A
4,UNM,29.250,7.55,N/A
5,HRB,24.100,7.99,N/A
6,CE,156.150,8.7,N/A
7,COO,423.700,8.89,N/A
8,PGR,99.602,9.02,N/A
9,KIM,22.080,9.14,N/A


## Calculating the Number of Shares to Buy

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:10000000000000


We can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,657.850,5.06,304020673
1,CINF,119.060,6.36,1679825298
2,ZION,54.000,6.65,3703703703
3,AFL,54.200,7.28,3690036900
4,UNM,29.250,7.55,6837606837
5,HRB,24.100,7.99,8298755186
6,CE,156.150,8.7,1280819724
7,COO,423.700,8.89,472032098
8,PGR,99.602,9.02,2007991807
9,KIM,22.080,9.14,9057971014


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, firms that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']

# EV/EBITDA
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. We'll use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,154.36,N/A,51.22,N/A,9.36,N/A,7.88,N/A,31.254935,N/A,14.963328,N/A,N/A
1,AAL,21.54,N/A,-1.48,N/A,-1.71,N/A,1.06,N/A,-3.651189,N/A,2.971237,N/A,N/A
2,AAP,207.00,N/A,22.72,N/A,3.84,N/A,1.26,N/A,11.495939,N/A,2.877254,N/A,N/A
3,AAPL,141.25,N/A,30.95,N/A,33.32,N/A,7.32,N/A,23.367052,N/A,17.797542,N/A,N/A
4,ABBV,116.75,N/A,39.7,N/A,14.57,N/A,4.14,N/A,12.685348,N/A,8.325723,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,118.19,N/A,31.71,N/A,-4.49,N/A,5.99,N/A,22.523088,N/A,10.018360,N/A,N/A
501,ZBH,168.23,N/A,60.72,N/A,2.78,N/A,4.84,N/A,28.336364,N/A,8.078262,N/A,N/A
502,ZBRA,534.17,N/A,44.17,N/A,11.97,N/A,6.21,N/A,33.291994,N/A,13.796000,N/A,N/A
503,ZION,55.53,N/A,6.5,N/A,1.19,N/A,2.5,N/A,5.258303,N/A,2.471078,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

We can use pandas' `isnull` method to identify missing data:

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,249.900,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,286.890,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.11,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,29.690,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.480,N/A,14.42,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,63.830,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,59.120,N/A,32.49,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,36.200,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,194.120,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2616.050,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this project, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [16]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.825743
1      0.120792
2      0.417822
3      0.570297
4      0.734653
         ...   
500    0.584158
501    0.869307
502    0.788119
503    0.130693
504    0.831683
Name: PE Percentile, Length: 505, dtype: object
0      0.790099
1      0.083168
2      0.516832
3      0.958416
4      0.869307
         ...   
500    0.081188
501    0.389109
502    0.831683
503    0.129703
504    0.932673
Name: PB Percentile, Length: 505, dtype: object
0      0.805941
1       0.10099
2      0.139604
3      0.786139
4      0.558416
         ...   
500    0.691089
501    0.613861
502    0.736634
503    0.358416
504    0.908911
Name: PS Percentile, Length: 505, dtype: object
0      0.831683
1      0.037624
2      0.283168
3       0.69901
4      0.344554
         ...   
500    0.671287
501    0.794059
502    0.881188
503    0.053465
504     0.89901
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      0.821782
1      0.130693
2      0.122772
3      0.875248
4      0.493069
         ...   
5

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,154.36,N/A,51.22,0.825743,9.36,0.790099,7.88,0.805941,31.254935,0.831683,14.963328,0.821782,N/A
1,AAL,21.54,N/A,-1.48,0.120792,-1.71,0.083168,1.06,0.10099,-3.651189,0.037624,2.971237,0.130693,N/A
2,AAP,207.00,N/A,22.72,0.417822,3.84,0.516832,1.26,0.139604,11.495939,0.283168,2.877254,0.122772,N/A
3,AAPL,141.25,N/A,30.95,0.570297,33.32,0.958416,7.32,0.786139,23.367052,0.69901,17.797542,0.875248,N/A
4,ABBV,116.75,N/A,39.70,0.734653,14.57,0.869307,4.14,0.558416,12.685348,0.344554,8.325723,0.493069,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,118.19,N/A,31.71,0.584158,-4.49,0.081188,5.99,0.691089,22.523088,0.671287,10.018360,0.631683,N/A
501,ZBH,168.23,N/A,60.72,0.869307,2.78,0.389109,4.84,0.613861,28.336364,0.794059,8.078262,0.481188,N/A
502,ZBRA,534.17,N/A,44.17,0.788119,11.97,0.831683,6.21,0.736634,33.291994,0.881188,13.796000,0.772277,N/A
503,ZION,55.53,N/A,6.50,0.130693,1.19,0.129703,2.50,0.358416,5.258303,0.053465,2.471078,0.10099,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [17]:
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,154.36,N/A,51.22,0.825743,9.36,0.790099,7.88,0.805941,31.254935,0.831683,14.963328,0.821782,0.81505
1,AAL,21.54,N/A,-1.48,0.120792,-1.71,0.083168,1.06,0.10099,-3.651189,0.037624,2.971237,0.130693,0.094653
2,AAP,207.00,N/A,22.72,0.417822,3.84,0.516832,1.26,0.139604,11.495939,0.283168,2.877254,0.122772,0.29604
3,AAPL,141.25,N/A,30.95,0.570297,33.32,0.958416,7.32,0.786139,23.367052,0.69901,17.797542,0.875248,0.777822
4,ABBV,116.75,N/A,39.70,0.734653,14.57,0.869307,4.14,0.558416,12.685348,0.344554,8.325723,0.493069,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,118.19,N/A,31.71,0.584158,-4.49,0.081188,5.99,0.691089,22.523088,0.671287,10.018360,0.631683,0.531881
501,ZBH,168.23,N/A,60.72,0.869307,2.78,0.389109,4.84,0.613861,28.336364,0.794059,8.078262,0.481188,0.629505
502,ZBRA,534.17,N/A,44.17,0.788119,11.97,0.831683,6.21,0.736634,33.291994,0.881188,13.796000,0.772277,0.80198
503,ZION,55.53,N/A,6.50,0.130693,1.19,0.129703,2.50,0.358416,5.258303,0.053465,2.471078,0.10099,0.154653


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [18]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.930,N/A,7.86,0.134653,0.5494,0.087129,0.4374,0.021782,3.636704,0.047525,0.426214,0.009901,0.060198
1,MCK,194.300,N/A,-7.15,0.10099,-1478.1000,0.00396,0.1290,0.009901,7.311462,0.089109,2.505043,0.10495,0.061782
2,AIV,7.082,N/A,-76.66,0.023762,1.9400,0.283168,-31.0600,0.00198,-43.573876,0.007921,-52.714763,0.005941,0.064554
3,L,57.450,N/A,-404.59,0.005941,0.8489,0.09901,1.0300,0.09802,7.943642,0.118812,0.991182,0.027723,0.069901
4,AIG,49.334,N/A,-11.39,0.087129,0.6607,0.091089,0.9519,0.085149,5.873230,0.061386,0.931157,0.025743,0.070099
5,HFC,34.780,N/A,-36.31,0.049505,1.0600,0.111881,0.4776,0.025743,8.225653,0.136634,2.930634,0.126733,0.090099
6,AAL,21.540,N/A,-1.48,0.120792,-1.7100,0.083168,1.0600,0.10099,-3.651189,0.037624,2.971237,0.130693,0.094653
7,ABC,117.707,N/A,-6.49,0.10495,-84.2200,0.009901,0.1231,0.007921,8.519181,0.144554,4.149660,0.217822,0.09703
8,BA,250.236,N/A,-11.76,0.085149,-7.8900,0.037624,2.5700,0.366337,-19.873689,0.019802,-43.524222,0.007921,0.103366
9,HPQ,30.530,N/A,11.20,0.182178,-10.8800,0.031683,0.6053,0.043564,7.990514,0.122772,3.197889,0.144554,0.10495


## Calculating the Number of Shares to Buy


In [19]:
portfolio_input()

Enter the value of your portfolio:1000000000000000


In [32]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][row])

C:\Users\sahus\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,29.930,668225860340,7.86,0.134653,0.5494,0.087129,0.4374,0.021782,3.636704,0.047525,0.426214,0.009901,0.060198
1,MCK,194.300,102933607822,-7.15,0.10099,-1478.1000,0.00396,0.1290,0.009901,7.311462,0.089109,2.505043,0.10495,0.061782
2,AIV,7.082,2824060999717,-76.66,0.023762,1.9400,0.283168,-31.0600,0.00198,-43.573876,0.007921,-52.714763,0.005941,0.064554
3,L,57.450,348128807658,-404.59,0.005941,0.8489,0.09901,1.0300,0.09802,7.943642,0.118812,0.991182,0.027723,0.069901
4,AIG,49.334,405399927028,-11.39,0.087129,0.6607,0.091089,0.9519,0.085149,5.873230,0.061386,0.931157,0.025743,0.070099
5,HFC,34.780,575043128234,-36.31,0.049505,1.0600,0.111881,0.4776,0.025743,8.225653,0.136634,2.930634,0.126733,0.090099
6,AAL,21.540,928505106778,-1.48,0.120792,-1.7100,0.083168,1.0600,0.10099,-3.651189,0.037624,2.971237,0.130693,0.094653
7,ABC,117.707,169913429107,-6.49,0.10495,-84.2200,0.009901,0.1231,0.007921,8.519181,0.144554,4.149660,0.217822,0.09703
8,BA,250.236,79924551223,-11.76,0.085149,-7.8900,0.037624,2.5700,0.366337,-19.873689,0.019802,-43.524222,0.007921,0.103366
9,HPQ,30.530,655093350802,11.20,0.182178,-10.8800,0.031683,0.6053,0.043564,7.990514,0.122772,3.197889,0.144554,0.10495


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization.

In [33]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric


In [34]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [35]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output


In [36]:
writer.save()